In [26]:
from pathlib import Path
import random
import pandas as pd

import IMP
import IMP.atom

import sys
sys.path.append("../../../src")
from weights import get_random_w_mat

In [39]:
pdb_file = Path(Path.home(), "Documents/xray/dev/43_phenix_ensemble_refinement/data/pdbs/3k0m.pdb")

n_decoys = 1000

m = IMP.Model()
hs = IMP.atom.read_multimodel_pdb(str(pdb_file), m, IMP.atom.ATOMPDBSelector())

In [40]:
for h in hs:
    for pid in IMP.atom.Selection(h).get_selected_particles():
        at = IMP.atom.Atom(m, pid)
        at.set_temperature_factor(15)

In [41]:
for i in range(n_decoys):
    hs_native = random.sample(hs, 2)
    IMP.atom.write_multimodel_pdb(hs_native, "../data/pdbs/{}.pdb".format(i))

In [43]:
decoy_df = pd.DataFrame()
for i in range(n_decoys):
    w_mat = get_random_w_mat(2,2)

    decoy_df.loc[i, "pdb"] = "/wynton/home/sali/mhancock/xray/dev/48_decoys/data/pdbs/{}.pdb".format(i)

    for state in range(2):
        for cond in range(2):
            decoy_df.loc[i, "w_{}_{}".format(state, cond)] = w_mat[state, cond]



In [44]:
decoy_df.to_csv("../data/csvs/decoys.csv")

#### correct structure with wrong weights

In [46]:
decoy_df = pd.read_csv("../data/csvs/decoys.csv", index_col=0)
decoy_df["pdb"] = "/wynton/home/sali/mhancock/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb"
decoy_df.to_csv("../data/csvs/decoys_w.csv")

#### wrong structure with correct weights

In [47]:
decoy_df = pd.read_csv("../data/csvs/decoys.csv", index_col=0)
decoy_df["w_0_0"] = .9
decoy_df["w_0_1"] = .1
decoy_df["w_1_0"] = .1
decoy_df["w_1_1"] = .9
decoy_df.to_csv("../data/csvs/decoys_c.csv")